In [4]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 32.3 MB/s eta 0:00:00


In [5]:
!pip install chromedriver-autoinstaller


In [6]:
# Import relevant libraries
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as expected_conditions

# --- Options Setup ---
options = webdriver.ChromeOptions()

user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.117 Safari/537.36"

options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument(f"user-agent={user_agent}")
options.add_argument("--window-size=1920,1080") # used to see all hosting plans at once


In [ ]:
!which chromedriver
!which chromium
!which chromium-browser
!chromedriver --version
!chromium --version || true
!chromium-browser --version || true

/usr/bin/chromedriver
/usr/bin/chromium-browser

Command '/usr/bin/chromedriver' requires the chromium snap to be installed.
Please install it with:

snap install chromium

/bin/bash: line 1: chromium: command not found

Command '/usr/bin/chromium-browser' requires the chromium snap to be installed.
Please install it with:

snap install chromium



In [9]:
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get update -y
!apt-get install -y ./google-chrome-stable_current_amd64.deb

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,572 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,633 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,205 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,860 kB]
Get:14 http:/

In [10]:
!pip -q install webdriver-manager


from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

chrome_options = Options()
chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--window-size=1920,1080")

# point to real Chrome binary
chrome_options.binary_location = "/usr/bin/google-chrome"

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

driver.get("https://books.toscrape.com/")
print(driver.current_url)

https://books.toscrape.com/


In [11]:
html_source = driver.page_source
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_source, 'html.parser')
books = soup.find_all('article', class_='product_pod')
len(books)

20

In [12]:
data = []

for book in books:
    title = book.h3.a["title"]
    price = book.find("p", class_="price_color").text
    availability = book.find("p", class_="instock availability").text.strip()

    # Rating is in class name like: "star-rating Three"
    rating_class = book.find("p", class_="star-rating")["class"]
    rating = rating_class[1]  # e.g. 'Three'

    data.append({
        "Title": title,
        "Price": price,
        "Availability": availability,
        "Rating": rating
    })
print(data)

[{'Title': 'A Light in the Attic', 'Price': '£51.77', 'Availability': 'In stock', 'Rating': 'Three'}, {'Title': 'Tipping the Velvet', 'Price': '£53.74', 'Availability': 'In stock', 'Rating': 'One'}, {'Title': 'Soumission', 'Price': '£50.10', 'Availability': 'In stock', 'Rating': 'One'}, {'Title': 'Sharp Objects', 'Price': '£47.82', 'Availability': 'In stock', 'Rating': 'Four'}, {'Title': 'Sapiens: A Brief History of Humankind', 'Price': '£54.23', 'Availability': 'In stock', 'Rating': 'Five'}, {'Title': 'The Requiem Red', 'Price': '£22.65', 'Availability': 'In stock', 'Rating': 'One'}, {'Title': 'The Dirty Little Secrets of Getting Your Dream Job', 'Price': '£33.34', 'Availability': 'In stock', 'Rating': 'Four'}, {'Title': 'The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull', 'Price': '£17.93', 'Availability': 'In stock', 'Rating': 'Three'}, {'Title': 'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics', 'P

In [16]:
import pandas as pd
df = pd.DataFrame(data)
df.to_csv('books.cvs', index = False)
print(df.head())

                                   Title   Price Availability Rating
0                   A Light in the Attic  £51.77     In stock  Three
1                     Tipping the Velvet  £53.74     In stock    One
2                             Soumission  £50.10     In stock    One
3                          Sharp Objects  £47.82     In stock   Four
4  Sapiens: A Brief History of Humankind  £54.23     In stock   Five


In [17]:
driver.quit()